In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
# generate numpy array from correlation matrix text file produced by run_30_pp_roi.tcsh
matrix_post = np.genfromtxt('sub-Linus_postdrug_3mm_smoothing.netcc', skip_header=6)
matrix_pre = np.genfromtxt('sub-Linus_predrug_3mm_smoothing.netcc', skip_header=6)

In [3]:
for i in range(len(matrix_post)):
    matrix_post[i,i] = np.nan
    matrix_pre[i,i] = np.nan

In [4]:
#slice matrix into parts that make sense for our analysis
# CL_CL: correlation within left hemisphere
# CR_CR: correlation within right hemisphere
# CL_CR: correlation across left and right hemisphere
# CL_CL: correlation across right and left hemisphere

slicer = int(len(matrix_post)/2)

CL_CL_matrix_pre = matrix_pre[:slicer,:slicer]
#print(CL_CL_matrix)
CR_CR_matrix_pre = matrix_pre[slicer:,slicer:]
#print(CR_CR_matrix)
CL_CR_matrix_pre = matrix_pre[:slicer,slicer:]
#CL_CR_matrix
CR_CL_matrix_pre = matrix_pre[slicer:,:slicer]
#CR_CL_matrix

CL_CL_matrix_post = matrix_post[:slicer,:slicer]

CR_CR_matrix_post = matrix_post[slicer:,slicer:]

CL_CR_matrix_post = matrix_post[:slicer,slicer:]

CR_CL_matrix_post = matrix_post[slicer:,:slicer]

In [5]:
#flatten the matrices because otherwise you get separate t-tests for every column with the ttest_rel function
CL_CL_matrix_pre_1d = CL_CL_matrix_pre.flatten()
CR_CR_matrix_pre_1d = CR_CR_matrix_pre.flatten()
CL_CR_matrix_pre_1d = CL_CR_matrix_pre.flatten()
CL_CL_matrix_post_1d = CL_CL_matrix_post.flatten()
CR_CR_matrix_post_1d = CR_CR_matrix_post.flatten()
CL_CR_matrix_post_1d = CL_CR_matrix_post.flatten()

In [6]:
#test for significance in CL-CL, CR-CR and CL-CR correlations
print(stats.ttest_rel(CL_CL_matrix_pre_1d, CL_CL_matrix_post_1d, nan_policy = 'omit'))
print(stats.ttest_rel(CR_CR_matrix_pre_1d, CR_CR_matrix_post_1d, nan_policy = 'omit'))
print(stats.ttest_rel(CL_CR_matrix_pre_1d, CL_CR_matrix_post_1d, nan_policy = 'omit'))

TtestResult(statistic=8.844439465508431, pvalue=1.2058856138850762e-16, df=271)
TtestResult(statistic=13.675500160768747, pvalue=9.882439061841118e-33, df=271)


TtestResult(statistic=12.060888691901923, pvalue=2.1736018808173026e-27, df=288)

In [7]:
#isolate only homotopic pairs
homotopic_array_pre = np.zeros(slicer)
homotopic_array_post = np.zeros(slicer)

for i in range(slicer):
    homotopic_array_pre[i] = matrix_pre[i, slicer+i]
    homotopic_array_post[i] = matrix_post[i, slicer+i]

In [8]:
#test for significance in correlation of homotopic pairs across hemispheres
print(stats.ttest_rel(homotopic_array_pre, homotopic_array_post, nan_policy = 'omit'))

TtestResult(statistic=3.7073901787816768, pvalue=0.001912288273868027, df=16)

In [9]:
#isolate only non-homotopic pairs
non_homotopic_array_pre = np.ma.array(CL_CR_matrix_pre, mask=False)
non_homotopic_array_post = np.ma.array(CL_CR_matrix_post, mask=False) 

for i in range(slicer):
    non_homotopic_array_pre.mask[i, i] = True
    non_homotopic_array_post.mask[i, i] = True

In [10]:
flat_pre = non_homotopic_array_pre.flatten()
flat_post = non_homotopic_array_post.flatten()

In [11]:
#test for significance of non-homotopic pairs across hemispheres
print(stats.ttest_rel(flat_pre, flat_post, nan_policy = 'omit'))

TtestResult(statistic=11.514947319549307, pvalue=3.0577517590147487e-25, df=271)